# Bootstrapping user preferences



Similarly to (Zhu et al., 2021), we'll evaluate a dynamic recommendation system using:

1. An oracle preference model (Generated in notebook 00-preference_model)
2. A position bias examination model
3. Bootstrapped user preferences

Step 3 is important so that we're simulating exposing users to fresh items, using their previous preferences (provided by step 1) and their examination bias (step 2) to determine a sinthethical rating matrix dataset that we'll use to compare different calibration approaches as the user gets exposed to more and more items.

We'll also simulate the timestamp of user interaction, with the following methodology:

1. We'll analyze the average delta in timestamps between consecutive ratings per user in the same dataset that the preference model was trained on (e.g: movielens) (present in data/movielens-1m/avg_str_time_dff_per_user.csv)
2. Assuming that the time_delta is normally distributed (with the mean and standart deviation being defined from the dataset), we'll randomly generate a delta_t for each user. the time_0 will be the timestamp when we start generating the user preferences.

In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
import numpy as np
import math
import random

In [3]:
import sys
import os
from pathlib import Path

sys.path.append('..')

In [4]:
from constants import ML_1M_FILLED_PATH

In [5]:
def get_user_preference_for_item(user, item, matrix):
    user_ratings = matrix[matrix["user"] == user]
    return user_ratings[user_ratings["item"] == item].rating.item()

In [6]:
def click_model(k):
    lambda_k = 1/math.log(k+1,2)
    examination_probability = random.random()
    if examination_probability <= lambda_k:
        return True
    return False

In [7]:
def get_inverse_propensity_click_score(position):
    # Given a click position, this funtion returns the invense propensity, 
    # usefull to debias the data later.
    return - 1/math.log(position+1,2)

In [8]:
def get_user_feedback_for_item(user, item ,k, matrix):
    preference = get_user_preference_for_item(user, item, matrix)
    observed = click_model(k)
    relevant = bool(preference)
    should_click = observed and relevant
    if (should_click):
        feedback = 1
        clicked_at = k
    else:
        if (observed):
            # Case where an item was observed, but isn´t relevant -> negative example for BPR
            feedback = 0
        else:
            # Case where an item was neither observed or relevant -> we will ignore this training instance in this loop
            feedback = None
        clicked_at = None
        
    # If user clicked the item, record the position it was in
    # feedback = 1 if user examined and clicked, 0 if user examined and not clicked,
    # None if otherwise
    return user, item, feedback, clicked_at

In [9]:
def map_recommendation_to_feedback(user, rec_list, matrix):
    return [get_user_feedback_for_item(user, item, idx+1, matrix) for idx, item in enumerate(rec_list)]

In [10]:
def random_rec(items, u, k, D):
    user_history = set(D[D["user"] == u]["item"])
    candidate_items = list(set(items) - user_history)
    return random.sample(candidate_items, k)

In [11]:
ML_1M_FILLED_PATH


'data/simulation/movielens_1m_sinthetically_filled.csv'

In [12]:
pd.read_csv('../data/simulation/movielens_1m_sinthetically_filled.csv')

,Unnamed: 0,user,item,Rating
0,0,1,1193,1
1,1,1,661,0
2,2,1,914,0
3,3,1,3408,1
4,4,1,2355,1
...,...,...,...,...
19574584,19574584,6040,2258,0
19574585,19574585,6040,2845,0
19574586,19574586,6040,3607,1
19574587,19574587,6040,690,0


In [19]:
preference_matrix = pd.read_csv(f"../{ML_1M_FILLED_PATH}").drop(columns=["Unnamed: 0"])
preference_matrix = preference_matrix.rename(columns={"Rating": "rating"})

In [20]:

preference_matrix

,user,item,rating
0,1,1193,1
1,1,661,0
2,1,914,0
3,1,3408,1
4,1,2355,1
...,...,...,...
19574584,6040,2258,0
19574585,6040,2845,0
19574586,6040,3607,1
19574587,6040,690,0


In [21]:
unique_users = list(preference_matrix["user"].unique())

unique_items = list(preference_matrix["item"].unique())

In [22]:
click_matrix = pd.DataFrame(columns=["user", "item", "clicked_and_examined", "clicked_at"])

1757177191.239523

In [26]:
pd.read_csv("../data/movielens-1m/avg_std_time_diff_per_user.csv")

,userId,avg_timestamp_diff,std_timestamp_diff
0,1,10083.307692,7.166287e+04
1,2,16.015625,2.070627e+01
2,3,29.720000,5.237711e+01
3,5,27.984772,7.034523e+01
4,6,42.057143,7.127710e+01
...,...,...,...
5284,6035,10.014337,1.938646e+01
5285,6036,51.687711,1.153505e+03
5286,6037,461.905473,5.807615e+03
5287,6039,433.368852,4.636566e+03


In [28]:
user_to_time_delta = pd.read_csv("../data/movielens-1m/avg_std_time_diff_per_user.csv").set_index("userId")

In [46]:
user_to_time_delta[user_to_time_delta["avg_timestamp_diff"] < 0]

,avg_timestamp_diff,std_timestamp_diff
userId,,


In [47]:
from scipy.stats import lognorm

userToDistribution = {
    user: lognorm(s=row["std_timestamp_diff"], scale=np.exp(row["avg_timestamp_diff"]))
    for user, row in user_to_time_delta.iterrows()
}


/var/folders/q8/hch9701n0gg0qyj291ylk_140000gn/T/ipykernel_96464/205468019.py:4: RuntimeWarning: overflow encountered in exp
  user: lognorm(s=row["std_timestamp_diff"], scale=np.exp(row["avg_timestamp_diff"]))


In [48]:
userToDistribution

{1: <scipy.stats._distn_infrastructure.rv_continuous_frozen at 0x1238092e0>,
 2: <scipy.stats._distn_infrastructure.rv_continuous_frozen at 0x123796c10>,
 3: <scipy.stats._distn_infrastructure.rv_continuous_frozen at 0x123809550>,
 5: <scipy.stats._distn_infrastructure.rv_continuous_frozen at 0x123796550>,
 6: <scipy.stats._distn_infrastructure.rv_continuous_frozen at 0x1238095b0>,
 7: <scipy.stats._distn_infrastructure.rv_continuous_frozen at 0x123809370>,
 8: <scipy.stats._distn_infrastructure.rv_continuous_frozen at 0x1237c90a0>,
 9: <scipy.stats._distn_infrastructure.rv_continuous_frozen at 0x123809dc0>,
 10: <scipy.stats._distn_infrastructure.rv_continuous_frozen at 0x1237c9340>,
 11: <scipy.stats._distn_infrastructure.rv_continuous_frozen at 0x1237a1af0>,
 13: <scipy.stats._distn_infrastructure.rv_continuous_frozen at 0x1237c9250>,
 15: <scipy.stats._distn_infrastructure.rv_continuous_frozen at 0x1237a1be0>,
 16: <scipy.stats._distn_infrastructure.rv_continuous_frozen at 0x1237c9

In [ ]:
# Problema: a variavle aleatoria pode ser negativa
userToDistribution[1].rvs(size=1)

/Users/caio.andrade/miniforge3/envs/mestrado/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:6478: RuntimeWarning: overflow encountered in exp
  return np.exp(s * random_state.standard_normal(size))


array([inf])

In [31]:
userToDistribution


{1: -7387.867667226157,
 2: 4.773049268923112,
 3: 16.65453789252306,
 5: 56.35468454160707,
 6: 24.31830695868401,
 7: 26.967292923439363,
 8: 276.2715632429951,
 9: 0.6863025757141514,
 10: 76810.92873087742,
 11: 51921.694596079986,
 13: 37.5635670123692,
 15: 17661.599009332313,
 16: 34.19984178802906,
 17: 14.889263743812762,
 18: 23.017495511723855,
 19: 33511.1052695285,
 22: -433496.9228925941,
 23: -634629.7240321985,
 24: 605166.0797594527,
 25: 743.2308536092041,
 26: -1193.599986786944,
 27: 33.20497196367559,
 28: -98813.83825217011,
 29: -660.0182163643076,
 30: -28.3246029683423,
 31: -13.74946275530147,
 32: -337.2786732789897,
 33: 13999.411758975042,
 34: 41.411454477033494,
 35: 12022.329345311258,
 36: -1105503.363578415,
 37: 2937.549245305467,
 38: -40.15677637072877,
 39: -7.5385585186129696,
 40: -31.210256457724796,
 42: 10.65296596847389,
 44: 1923061.0581009267,
 45: -15.912129389893908,
 46: 41.42610562921139,
 48: -16835.686097611415,
 49: -16875.0418874662

In [29]:
user_to_time_delta

,avg_timestamp_diff,std_timestamp_diff
userId,,
1,10083.307692,7.166287e+04
2,16.015625,2.070627e+01
3,29.720000,5.237711e+01
5,27.984772,7.034523e+01
6,42.057143,7.127710e+01
...,...,...
6035,10.014337,1.938646e+01
6036,51.687711,1.153505e+03
6037,461.905473,5.807615e+03


In [ ]:
def bootstrap_clicks(D, unique_users, unique_items, preference_matrix, k=20, rounds=10):
    """
    Given unique users and unique items, recommend up to k items to every user
    using a preference matrix as a relevancy model and using a click model
    to simulate probability of user examinating an item.

    Feedback signal will be fed to the D matrix.

    We run the boostrap process for a total of an arbitrary number of rounds,
    in order to ensure enough feedback data to train a model.
    """
    new_df = D.copy()#pd.DataFrame(columns=["user", "item", "clicked_and_examined", "clicked_at"])
    initial_time = pd.Timestamp.now()
    for round in range(rounds):
        rows_to_append = []
        for user in tqdm(unique_users, desc=f"Processing users (round {round+1}/{rounds})..."):
            recs = random_rec(unique_items, user, k, new_df)
            row = map_recommendation_to_feedback(user, recs, preference_matrix)
            rows_to_append.extend(row)
        round_df = pd.DataFrame(rows_to_append, columns=new_df.columns)
        new_df = pd.concat([new_df, round_df], ignore_index=True)
    return pd.concat([D, new_df])

In [ ]:
%%time
click_matrix = bootstrap_clicks(click_matrix, unique_users, unique_items, preference_matrix)

Processing users (round 1/10)...: 100%|██████████| 6040/6040 [18:02<00:00,  5.58it/s]
/tmp/ipykernel_7989/1541821497.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, round_df], ignore_index=True)
Processing users (round 6/10)...:  43%|████▎     | 2575/6040 [08:45<12:02,  4.80it/s]

In [ ]:
click_matrix

,user,item,relevancy,clicked_at
0,1,3183,1.0,1.0
1,1,1375,NaN,NaN
2,1,2162,NaN,NaN
3,1,353,NaN,NaN
4,1,2519,NaN,NaN
...,...,...,...,...
120795,6040,3248,NaN,NaN
120796,6040,3937,0.0,NaN
120797,6040,3423,NaN,NaN
120798,6040,3246,NaN,NaN


In [ ]:
click_matrix.groupby(["relevancy", "clicked_at"], dropna=False).size()

relevancy  clicked_at
0.0        NaN           28588
1.0        1.0            1988
           2.0            1251
           3.0             960
           4.0             842
           5.0             745
           6.0             735
           7.0             652
           8.0             617
           9.0             560
           10.0            540
           11.0            573
           12.0            518
           13.0            508
           14.0            519
           15.0            481
           16.0            433
           17.0            492
           18.0            481
           19.0            423
           20.0            455
NaN        NaN           78439
dtype: int64

In [ ]:
click_matrix.to_csv("../data/simulation/sinthetic_data.csv", index=False)

In [ ]:
click_matrix = pd.read_csv("../data/simulation/sinthetic_data.csv")

In [ ]:
click_matrix[click_matrix["relevancy"].notna()]["relevancy"].value_counts(dropna=False)

relevancy
0.0    28588
1.0    13773
Name: count, dtype: int64